## Import Relevant Libraries

In [95]:
import pandas as pd
import numpy as np
import nltk

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

## Load Data

In [96]:
df_jobs_final = pd.read_csv("Combined_Jobs_Final.csv")

## Exploratory Data Analysis

In [97]:
df_jobs_final.head()

,Job.ID,Provider,Status,Slug,Title,Position,Company,City,State.Name,State.Code,...,Industry,Job.Description,Requirements,Salary,Listing.Start,Listing.End,Employment.Type,Education.Required,Created.At,Updated.At
0,111,1,open,palo-alto-ca-tacolicious-server,Server @ Tacolicious,Server,Tacolicious,Palo Alto,California,CA,...,Food and Beverages,Tacolicious' first Palo Alto store just opened...,NaN,8.00,NaN,NaN,Part-Time,NaN,2013-03-12 02:08:28 UTC,2014-08-16 15:35:36 UTC
1,113,1,open,san-francisco-ca-claude-lane-kitchen-staff-chef,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,California,CA,...,Food and Beverages,\r\n\r\nNew French Brasserie in S.F. Financia...,NaN,0.00,NaN,NaN,Part-Time,NaN,2013-04-12 08:36:36 UTC,2014-08-16 15:35:36 UTC
2,117,1,open,san-francisco-ca-machka-restaurants-corp-barte...,Bartender @ Machka Restaurants Corp.,Bartender,Machka Restaurants Corp.,San Francisco,California,CA,...,Food and Beverages,We are a popular Mediterranean wine bar and re...,NaN,11.00,NaN,NaN,Part-Time,NaN,2013-07-16 09:34:10 UTC,2014-08-16 15:35:37 UTC
3,121,1,open,brisbane-ca-teriyaki-house-server,Server @ Teriyaki House,Server,Teriyaki House,Brisbane,California,CA,...,Food and Beverages,● Serve food/drinks to customers in a profess...,NaN,10.55,NaN,NaN,Part-Time,NaN,2013-09-04 15:40:30 UTC,2014-08-16 15:35:38 UTC
4,127,1,open,los-angeles-ca-rosa-mexicano-sunset-kitchen-st...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset,Kitchen Staff/Chef,Rosa Mexicano - Sunset,Los Angeles,California,CA,...,Food and Beverages,"Located at the heart of Hollywood, we are one ...",NaN,10.55,NaN,NaN,Part-Time,NaN,2013-07-17 15:26:18 UTC,2014-08-16 15:35:40 UTC


In [98]:
df_jobs_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84090 entries, 0 to 84089
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Job.ID              84090 non-null  int64  
 1   Provider            84090 non-null  int64  
 2   Status              84090 non-null  object 
 3   Slug                84090 non-null  object 
 4   Title               84090 non-null  object 
 5   Position            84090 non-null  object 
 6   Company             81819 non-null  object 
 7   City                83955 non-null  object 
 8   State.Name          83919 non-null  object 
 9   State.Code          83919 non-null  object 
 10  Address             36 non-null     object 
 11  Latitude            84090 non-null  float64
 12  Longitude           84090 non-null  float64
 13  Industry            267 non-null    object 
 14  Job.Description     84034 non-null  object 
 15  Requirements        0 non-null      float64
 16  Sala

In [99]:
df_jobs_final.describe()

,Job.ID,Provider,Latitude,Longitude,Requirements,Salary
count,84090.000000,84090.000000,84090.000000,84090.000000,0.0,229.000000
mean,258490.774979,1.997063,37.967134,-92.151257,NaN,7.832227
std,52653.870401,0.056272,5.458651,17.412900,NaN,7.566016
min,3.000000,1.000000,-34.887672,-166.539760,NaN,0.000000
25%,250415.250000,2.000000,34.072600,-104.249780,NaN,0.000000
50%,271452.500000,2.000000,39.218300,-86.941440,NaN,8.000000
75%,293672.750000,2.000000,41.598965,-79.997460,NaN,10.550000
max,319174.000000,3.000000,71.294700,144.885800,NaN,58.000000


## Data Cleaning

In [100]:
df_jobs_final.isnull().sum()

Job.ID                    0
Provider                  0
Status                    0
Slug                      0
Title                     0
Position                  0
Company                2271
City                    135
State.Name              171
State.Code              171
Address               84054
Latitude                  0
Longitude                 0
Industry              83823
Job.Description          56
Requirements          84090
Salary                83861
Listing.Start           683
Listing.End             167
Employment.Type          10
Education.Required      267
Created.At                0
Updated.At                0
dtype: int64

In [101]:
# Replacing Nan Values

df_jobs_final['Company'] = df_jobs_final['Company'].replace(['Genesis Health Systems'], 'Genesis Health System')
df_jobs_final.loc[df_jobs_final.Company == 'CHI Payment Systems', 'City'] = 'Illinois'
df_jobs_final.loc[df_jobs_final.Company == 'Academic Year In America', 'City'] = 'Stamford'
df_jobs_final.loc[df_jobs_final.Company == 'CBS Healthcare Services and Staffing ', 'City'] = 'Urbandale'
df_jobs_final.loc[df_jobs_final.Company == 'Driveline Retail', 'City'] = 'Coppell'
df_jobs_final.loc[df_jobs_final.Company == 'Educational Testing Services', 'City'] = 'New Jersey'
df_jobs_final.loc[df_jobs_final.Company == 'Genesis Health System', 'City'] = 'Davennport'
df_jobs_final.loc[df_jobs_final.Company == 'Home Instead Senior Care', 'City'] = 'Nebraska'
df_jobs_final.loc[df_jobs_final.Company == 'St. Francis Hospital', 'City'] = 'New York'
df_jobs_final.loc[df_jobs_final.Company == 'Volvo Group', 'City'] = 'Washington'
df_jobs_final.loc[df_jobs_final.Company == 'CBS Healthcare Services and Staffing', 'City'] = 'Urbandale'

In [102]:
df_jobs_final.isnull().sum()

Job.ID                    0
Provider                  0
Status                    0
Slug                      0
Title                     0
Position                  0
Company                2271
City                      0
State.Name              171
State.Code              171
Address               84054
Latitude                  0
Longitude                 0
Industry              83823
Job.Description          56
Requirements          84090
Salary                83861
Listing.Start           683
Listing.End             167
Employment.Type          10
Education.Required      267
Created.At                0
Updated.At                0
dtype: int64

In [103]:
df_jobs_final['Employment.Type'].unique()

array(['Part-Time', 'Full-Time/Part-Time', 'Seasonal/Temp', 'Per Diem',
       'Intern', nan, 'Full-Time', 'Contract', 'Temporary/seasonal'],
      dtype=object)

In [104]:
df_jobs_final[pd.isnull(df_jobs_final['Employment.Type'])]

,Job.ID,Provider,Status,Slug,Title,Position,Company,City,State.Name,State.Code,...,Industry,Job.Description,Requirements,Salary,Listing.Start,Listing.End,Employment.Type,Education.Required,Created.At,Updated.At
10768,153197,3,open,san-francisco-ca-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,San Francisco,California,CA,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:46 UTC,2014-12-18 21:03:18 UTC
10769,153198,3,open,los-angeles-ca-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,Los Angeles,California,CA,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:47 UTC,2014-12-18 21:03:18 UTC
10770,153199,3,open,chicago-il-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,Chicago,Illinois,IL,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:48 UTC,2014-12-18 21:03:18 UTC
10771,153200,3,open,boston-ma-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,Boston,Massachusetts,MA,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:48 UTC,2014-12-18 21:03:18 UTC
10772,153201,3,open,ann-arbor-mi-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,Ann Arbor,Michigan,MI,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:49 UTC,2014-12-18 21:03:18 UTC
10773,153202,3,open,oklahoma-ok-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,Oklahoma,Oklahoma,OK,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:49 UTC,2014-12-18 21:03:18 UTC
10774,153203,3,open,omaha-ne-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,Omaha,Nebraska,NE,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:50 UTC,2014-12-18 21:03:18 UTC
10775,153204,3,open,lincoln-ne-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,Lincoln,Nebraska,NE,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:51 UTC,2014-12-18 21:03:18 UTC
10776,153205,3,open,minneapolis-mn-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,Minneapolis,Minnesota,MN,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:51 UTC,2014-12-18 21:03:18 UTC
10777,153206,3,open,st-paul-mn-uber-driving-partner,Driving Partner @ Uber,Driving Partner,Uber,St. Paul,Minnesota,MN,...,Transportation,Uber is changing the way the world moves. From...,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-17 06:08:52 UTC,2014-12-18 21:03:18 UTC


In [105]:
df_jobs_final['Employment.Type']=df_jobs_final['Employment.Type'].fillna('Full-Time/Part-Time')
df_jobs_final.isnull().sum()

Job.ID                    0
Provider                  0
Status                    0
Slug                      0
Title                     0
Position                  0
Company                2271
City                      0
State.Name              171
State.Code              171
Address               84054
Latitude                  0
Longitude                 0
Industry              83823
Job.Description          56
Requirements          84090
Salary                83861
Listing.Start           683
Listing.End             167
Employment.Type           0
Education.Required      267
Created.At                0
Updated.At                0
dtype: int64

## Data Preprocessing 

In [106]:
df_jobs_final["Text"] = df_jobs_final["Position"].map(str) + " " + df_jobs_final["Company"] +" "+ df_jobs_final["City"]+ " "+df_jobs_final['Employment.Type']+" "+df_jobs_final['Job.Description'] +" "+df_jobs_final['Title']
df_jobs_final_preprocessed = df_jobs_final[['Job.ID', 'Text', 'Title']]

In [107]:
df_jobs_final_preprocessed['Text'] = df_jobs_final_preprocessed['Text'].astype(str)
# df_jobs_final_preprocessed.loc[:, 'Text'] = df_jobs_final_preprocessed['Text'].astype(str)

df_jobs_final_preprocessed.head()

/var/folders/tm/s0hrvkhd7bb_g1qgmxfgsrx40000gn/T/ipykernel_6274/171349623.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jobs_final_preprocessed['Text'] = df_jobs_final_preprocessed['Text'].astype(str)


,Job.ID,Text,Title
0,111,Server Tacolicious Palo Alto Part-Time Tacolic...,Server @ Tacolicious
1,113,Kitchen Staff/Chef Claude Lane San Francisco P...,Kitchen Staff/Chef @ Claude Lane
2,117,Bartender Machka Restaurants Corp. San Francis...,Bartender @ Machka Restaurants Corp.
3,121,Server Teriyaki House Brisbane Part-Time ● Se...,Server @ Teriyaki House
4,127,Kitchen Staff/Chef Rosa Mexicano - Sunset Los ...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset


In [109]:


from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)


In [110]:
df_jobs_final_preprocessed.head()

,Job.ID,Text,Title
0,111,server tacolicious palo alto part time tacolic...,Server @ Tacolicious
1,113,kitchen staff chef claude lane san francisco p...,Kitchen Staff/Chef @ Claude Lane
2,117,bartender machka restaurants corp san francisc...,Bartender @ Machka Restaurants Corp.
3,121,server teriyaki house brisbane part time serve...,Server @ Teriyaki House
4,127,kitchen staff chef rosa mexicano sunset los an...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset


In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_jobid = tfidf_vectorizer.fit_transform((df_jobs_final_preprocessed['Text'])) 
tfidf_jobid

<84090x50755 sparse matrix of type '<class 'numpy.float64'>'
	with 8266022 stored elements in Compressed Sparse Row format>

In [112]:
df_job_views = pd.read_csv("Job_Views.csv")
df_job_views = df_job_views[['Applicant.ID', 'Job.ID', 'Position', 'Company','City']]
df_job_views["select_pos_com_city"] = df_job_views["Position"].map(str) + "  " + df_job_views["Company"] +"  "+ df_job_views["City"]
df_job_views['select_pos_com_city'] = df_job_views['select_pos_com_city'].map(str).apply(clean_txt)
df_job_views['select_pos_com_city'] = df_job_views['select_pos_com_city'].str.lower()
df_job_views = df_job_views[['Applicant.ID','select_pos_com_city']]
df_job_views.head()

,Applicant.ID,select_pos_com_city
0,10000,cashier valet need wallypark newark
1,10000,macys seasonal retail fragrance cashier garden...
2,10001,part time showroom sales cashier grizzly indus...
3,10002,event specialist part time advantage sales mar...
4,10002,bonefish kitchen staff bonefish grill greenville


In [113]:

df_experience = pd.read_csv("Experience.csv")
df_experience= df_experience[['Applicant.ID','Position.Name']]
df_experience['Position.Name'] = df_experience['Position.Name'].map(str).apply(clean_txt)
df_experience =  df_experience.sort_values(by='Applicant.ID')
df_experience = df_experience.fillna(" ")
df_experience = df_experience.groupby('Applicant.ID', sort=False)['Position.Name'].apply(' '.join).reset_index()
df_experience.head()

,Applicant.ID,Position.Name
0,2,volunteer writer uloop blog
1,3,market intern server prep cook
2,6,project assistant
3,8,deli clerk server cashier food prep order taker
4,11,cashier


In [114]:

df_position_of_interest =  pd.read_csv("Positions_Of_Interest.csv", sep=',')
df_position_of_interest = df_position_of_interest.sort_values(by='Applicant.ID')
df_position_of_interest = df_position_of_interest.drop('Updated.At', 1)
df_position_of_interest = df_position_of_interest.drop('Created.At', 1)
df_position_of_interest['Position.Of.Interest']=df_position_of_interest['Position.Of.Interest'].map(str).apply(clean_txt)
df_position_of_interest = df_position_of_interest.fillna(" ")
df_position_of_interest = df_position_of_interest.groupby('Applicant.ID', sort=True)['Position.Of.Interest'].apply(' '.join).reset_index()
df_position_of_interest.head()

/var/folders/tm/s0hrvkhd7bb_g1qgmxfgsrx40000gn/T/ipykernel_6274/247911197.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_position_of_interest = df_position_of_interest.drop('Updated.At', 1)
/var/folders/tm/s0hrvkhd7bb_g1qgmxfgsrx40000gn/T/ipykernel_6274/247911197.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_position_of_interest = df_position_of_interest.drop('Created.At', 1)


,Applicant.ID,Position.Of.Interest
0,96,server
1,153,barista host server sales rep customer service...
2,256,host production area sales rep customer servic...
3,438,customer service rep barista host server
4,568,receptionist customer service rep book keeper


In [115]:

df_jobs_experience = df_job_views.merge(df_experience, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
df_jobs_experience = df_jobs_experience.fillna(' ')
df_jobs_experience = df_jobs_experience.sort_values(by='Applicant.ID')

df_jobs_exp_position_of_interest = df_jobs_experience.merge(df_position_of_interest, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
df_jobs_exp_position_of_interest = df_jobs_exp_position_of_interest.fillna(' ')
df_jobs_exp_position_of_interest = df_jobs_exp_position_of_interest.sort_values(by='Applicant.ID')

df_jobs_exp_position_of_interest["text"] = df_jobs_exp_position_of_interest["select_pos_com_city"].map(str) + df_jobs_exp_position_of_interest["Position.Name"] +" "+ df_jobs_exp_position_of_interest["Position.Of.Interest"]

df_target_users= df_jobs_exp_position_of_interest[['Applicant.ID','text']]

df_target_users.columns = ['Applicant_id','text']

df_target_users['text'] = df_target_users['text'].apply(clean_txt)

df_target_users.head()

/var/folders/tm/s0hrvkhd7bb_g1qgmxfgsrx40000gn/T/ipykernel_6274/3296081758.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target_users['text'] = df_target_users['text'].apply(clean_txt)


,Applicant_id,text
0,2,volunteer writer uloop blog
1,3,market intern server prep cook
2,6,project assistant
3,8,deli clerk server cashier food prep order taker
4,11,cashier


In [116]:
# Identifing our test applicant 

applicant_id = 326
index = np.where(df_target_users['Applicant_id'] == applicant_id)[0][0]
test_applicant = df_target_users.iloc[[index]]
test_applicant

,Applicant_id,text
186,326,java developer


In [117]:

user_tfidf = tfidf_vectorizer.transform(test_applicant['text'])
cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_jobid)
output_cos_similarity_tfidf = list(cos_similarity_tfidf)

In [118]:
def get_recommendation(top, df_jobs_final_preprocessed, scores):
  recommendation = pd.DataFrame(columns = ['ApplicantID', 'JobID',  'title', 'score'])
  count = 0
  for i in top:
      recommendation.at[count, 'ApplicantID'] = applicant_id
      recommendation.at[count, 'JobID'] = df_jobs_final_preprocessed['Job.ID'][i]
      recommendation.at[count, 'title'] = df_jobs_final_preprocessed['Title'][i]
      recommendation.at[count, 'score'] =  scores[count]
      count += 1
  return recommendation

## Top Recomendations using TF-IDK

In [119]:
top = sorted(range(len(output_cos_similarity_tfidf)), key=lambda i: output_cos_similarity_tfidf[i], reverse=True)[:10]
list_scores = [output_cos_similarity_tfidf[i][0][0] for i in top]
get_recommendation(top,df_jobs_final_preprocessed, list_scores)

,ApplicantID,JobID,title,score
0,326,303112,Java Developer @ TransHire,0.749478
1,326,294684,Java Developer @ Kavaliro,0.740886
2,326,269922,Entry Level Java Developer / Jr. Java Develope...,0.737007
3,326,141831,Lead Java/J2EE Developer - Contract to Hire @ ...,0.671667
4,326,270171,Senior Java Developer - Contract to Hire - Gre...,0.645037
5,326,305264,Sr. Java Developer @ Paladin Consulting Inc,0.625532
6,326,309945,"Java Software Engineer @ iTech Solutions, Inc.",0.592291
7,326,245753,Java Administrator @ ConsultNet,0.530231
8,326,146640,Jr. Java Developer @ Paladin Consulting Inc,0.510534
9,326,150882,Java Consultant - Mobile Apps Development @ Co...,0.486789


In [120]:

count_vectorizer = CountVectorizer()

count_job_id = count_vectorizer.fit_transform((df_jobs_final_preprocessed['Text'])) #fitting and transforming the vector
count_job_id

<84090x50755 sparse matrix of type '<class 'numpy.int64'>'
	with 8266022 stored elements in Compressed Sparse Row format>

In [121]:

user_count = count_vectorizer.transform(test_applicant['text'])
cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_job_id)
output_cos_similarity_countv = list(cos_similarity_countv)

## Top Recomendations using CountVectorizer

In [122]:
top = sorted(range(len(output_cos_similarity_countv)), key=lambda i: output_cos_similarity_countv[i], reverse=True)[:10]
list_scores = [output_cos_similarity_countv[i][0][0] for i in top]
get_recommendation(top, df_jobs_final_preprocessed, list_scores)

,ApplicantID,JobID,title,score
0,326,303112,Java Developer @ TransHire,0.635001
1,326,294684,Java Developer @ Kavaliro,0.600245
2,326,269922,Entry Level Java Developer / Jr. Java Develope...,0.571726
3,326,141831,Lead Java/J2EE Developer - Contract to Hire @ ...,0.496907
4,326,270171,Senior Java Developer - Contract to Hire - Gre...,0.481757
5,326,309945,"Java Software Engineer @ iTech Solutions, Inc.",0.454673
6,326,305264,Sr. Java Developer @ Paladin Consulting Inc,0.406017
7,326,245753,Java Administrator @ ConsultNet,0.378968
8,326,150882,Java Consultant - Mobile Apps Development @ Co...,0.363216
9,326,146640,Jr. Java Developer @ Paladin Consulting Inc,0.323381


In [123]:

n_neighbors = 11
p = 2

KNN = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
KNN.fit(tfidf_jobid)
NNs = KNN.kneighbors(user_tfidf, return_distance=True)
NNs[0][0][1:]

array([0.71988065, 0.72524925, 0.81034882, 0.84257062, 0.86541138,
       0.90300462, 0.96929812, 0.98940985, 1.01312513, 1.08235252])

## Top Recomendations using KNN

In [124]:
top = NNs[1][0][1:]
index_score = NNs[0][0][1:]

get_recommendation(top, df_jobs_final_preprocessed, index_score)

,ApplicantID,JobID,title,score
0,326,294684,Java Developer @ Kavaliro,0.719881
1,326,269922,Entry Level Java Developer / Jr. Java Develope...,0.725249
2,326,141831,Lead Java/J2EE Developer - Contract to Hire @ ...,0.810349
3,326,270171,Senior Java Developer - Contract to Hire - Gre...,0.842571
4,326,305264,Sr. Java Developer @ Paladin Consulting Inc,0.865411
5,326,309945,"Java Software Engineer @ iTech Solutions, Inc.",0.903005
6,326,245753,Java Administrator @ ConsultNet,0.969298
7,326,146640,Jr. Java Developer @ Paladin Consulting Inc,0.98941
8,326,150882,Java Consultant - Mobile Apps Development @ Co...,1.013125
9,326,251696,Java Developer @ ConsultNet,1.082353
